설계방향
+ [기준 Text] 불러오기
+ **[대상 Text]** 불러오기(가격자료)
+ [기준 Text] 기준, **[대상 Text]** 유사도 계산 : Tf-idf
+ 인덱스 매칭 : 유사도 0.3 이상인 경우? $\to$ (검토 필요)
+ 매칭된 제품에 대해 데이터 정렬

In [19]:
import pandas as pd
from datetime import datetime, timedelta
import time 

# local only
import os
from google.cloud import bigquery
from skimage import io

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

path = 'C:/windows/Fonts/malgun.ttf'                           # 폰트:맑은고딕
font_name = fm.FontProperties(fname=path, size=24).get_name()  
plt.rc('font', family=font_name)

# 내용공유(e-mail)
import os
from os import listdir
from jinja2 import Template

import gspread

In [81]:
### 기준 Text 불러오기

# 기준 테이블 작성
df_item = pd.DataFrame()
df_item['제품/용량'] = [
                        '셀렉스 코어 프로틴 프로 / 304g x 1' ,
                        '하이뮨 프로틴 밸런스 / 304g x 1', 
                        '코어틴 그린 프로틴 / 280g x 1',
                        '롯데헬스원 초유 프로틴 365 / 280g x 1',
                        '칼로바이 라이프 프로틴 파우더 / 280g x 1',
                        '마이밀 뉴프로틴 산양유 단백질 / 340gx 1',
                        '마이밀 자연방목 프로틴 / 280g x 1',
                        
                        '셀렉스 코어 프로틴 프로 스틱 / 19g x 10',
                        '하이뮨 프로틴 밸런스 스틱 / 19g x 10',
#                         '마이밀 뉴프로틴 플레인 / 17g x 20',
                        '이너랩 마더플러스 초유 프로틴 / 40g x 7',

#                         '셀렉스 코어프로틴 프로 캔 304g + 스틱 19g x 10p, 1세트',
                        '하이뮨 프로틴 밸런스 스틱 19g x 20 + 보틀',

                        '셀렉스 스포츠 웨이 프로틴 파우더 초콜릿 / 627g x 1',
                        '셀렉스 프로핏 웨이 프로틴 파우더 복숭아 / 627g x 1',
#                         '라이프프로틴 스포츠 초코맛 / 646g x 1',
                        '라이프프로틴 스포츠 딸기맛 / 646g x 1',
#                         '테이크핏 단백질 초코맛 / 280g x 1',
    
                        '셀렉스 너츠바 / 50g x 6',
                        '마이밀 뉴프로틴바 / 21g x 12',
#                         '빙그레 크런치바 / 40g x 9',
                        '칼로바이 단백질바 / 35g x 10',
    
                        '셀렉스 프리바이오틱스 / 12.5g x 10',
                        '하이뮨 프리&프로바이오틱스 / 5g x 30',
    
                        '셀렉스 마시는 프로틴 / 125ml x 24',
                        '셀렉스 코어프로틴 오리지널 / 125ml x 14',
                        '하이뮨 마시는 프로틴 / 125ml x 12',
                        '하이뮨 마시는 프로틴 / 190ml x 24',
                        '마이밀 마시는 뉴프로틴 / 190ml x 16',
                        ]
df_item = df_item.set_index(['제품/용량'])

In [21]:
####################
### 대상 Text(가격자료) 불러오기
######

# 접속 정보 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./********************************.json"
client = bigquery.Client()

# rawdata 불러오기
sql = """SELECT A.category, A.PRODUCT, A.URL
         , FORMAT_DATETIME('%m/%d', CAST(A.DATE AS DATETIME)) as DATE
         , (case when trim(A.DELIVERY_TYPE)  ='기타' then '-' 
                 when trim(A.DELIVERY_TYPE)  ='로켓직구' then '-'
                 else trim(A.DELIVERY_TYPE) end) as DELIVERY_TYPE
         , A.PRICE_ORIGIN
         , A.PRICE_SALE 
         FROM `************************.online_price.price_coupang_v2` A
         where A.category in ('기타영양식', '복합 프로틴파우더', '기타헬스보조제', '초유', 'WPI 분리유청', '가르니시아')
         and A.CHANNEL = 'coupang'
         and A.SOLDOUT = '판매중'
         and FORMAT_DATETIME('%H', CAST(A.DATE AS DATETIME)) in ( '09' )
         and CAST(A.DATE AS DATETIME) between DATE_SUB(current_date(), INTERVAL 28 DAY) and DATE_SUB(current_date(), INTERVAL -1 DAY)"""
df1 = client.query(sql).to_dataframe()

In [82]:
df_item_list    = df_item.index.values.tolist()
df_product_list = list(df1.PRODUCT.drop_duplicates())

In [83]:
df_list = df_item_list + df_product_list

In [84]:
# Tfidf : real

from sklearn.feature_extraction.text import TfidfVectorizer

corpus = df_list
vect = TfidfVectorizer(min_df=1, stop_words="english")
tfidf = vect.fit_transform(corpus)
pairwise_similarity = tfidf * tfidf.T 

In [63]:
0.7

In [87]:
i = 9
df_cal = pd.DataFrame(pairwise_similarity.toarray()[i])[len(df_item_list):].reset_index(drop=True)
df_cal['PRODUCT'] = df_product_list

print('대상제품 : {}'.format(df_item_list[i]))
df_cal.sort_values(by=0, ascending=False).head(20)

대상제품 : 이너랩 마더플러스 초유 프로틴 / 40g x 7


,0,PRODUCT
1133,0.342798,"셀트리온 이너랩 헬시플러스 초유프로틴 네델란드 단백질 분말 락토페린, 40g, 7개"
848,0.260147,"롯데헬스원 초유 프로틴 365 280g, 롯데헬스원 초유 40g 7개+1개(총"
1104,0.248051,"셀트리온 이너랩 헬시플러스 초유프로틴 네델란드 단백질 분말 락토페린, 1개, 280g"
902,0.248051,"셀트리온 이너랩 헬시플러스 초유프로틴 네델란드 단백질 분말 락토페린, 280g, 1개"
188,0.204215,"셀트리온 마더 플러스 초유 프로틴 김호중 단백질, 40g, 2개"
1204,0.193940,"셀트리온 마더 플러스 초유 김호중 프로틴 단백질 파우더, 40g, 6개"
491,0.174370,"웰프로틴 초코, 40g, 8개"
1050,0.169525,"미트리 프로틴 초코바 크런치 40g, 04_프로틴 초코바 크런치 40g 2박스(40개)"
628,0.158349,"미트리 프로틴 초코바 아몬드 20p, 40g, 1박스"
1807,0.157640,"웰프로틴 파우더 그레인, 40g, 10개"


In [ ]:
# Tfidf : test1

from sklearn.feature_extraction.text import TfidfVectorizer

corpus = ["I'd like an apple", 
           "An apple a day keeps the doctor away", 
           "Never compare an apple to an orange",
           "I prefer scikit-learn to Orange", 
           "The scikit-learn docs are Orange and Blue"]
vect = TfidfVectorizer(min_df=1, stop_words="english")
tfidf = vect.fit_transform(corpus)
pairwise_similarity = tfidf * tfidf.T 

pairwise_similarity.toarray()

In [7]:
# Tfidf : test2

from sklearn.feature_extraction.text import TfidfVectorizer

corpus = ["셀렉스 프리바이오틱스 / 12.5g x 10", 
          '셀렉스 썬화이버 프리바이오틱스 스틱 12.5g X 10포, 단품, 단품', 
          '셀렉스 썬화이버 프리바이오틱스 스틱 12.5g 10포 x 1박스',
          '셀렉스 썬화이버 프리바이오틱스 스틱 12.5g 20포, 단일옵션',
          '셀렉스 코어 프로틴 너츠바 50g x 6p, 1개, 300g',
          '칼로바이 퍼펙트파워 프로틴바 단백질바 요거트맛, 35g, 10개',
          '셀렉스 썬화이버 프리바이오틱스 스틱 1박스 (12.5g X 10포), 단품',
          '셀렉스 썬화이버 프리바이오틱스 스틱 12.5g 10포 1박스, 상세페이지참조',
          '일동후디스 하이뮨 프리 프로바이오틱스, 30개, 5g',
          '마이밀 마시는 뉴프로틴, 190ml, 16개',
          '셀렉스 매일 마시는 프로틴, 125ml, 24개',
          '셀렉스 코어프로틴 음료 오리지널, 125ml, 14개',
          '셀렉스 코어프로틴 음료 오리지널, 14개입',
          '하이뮨 마시는 프로틴 밸런스, 125ml, 12개',
          '후디스 하이뮨 마시는 프로틴 밸런스, 125ml, 12개',]
vect = TfidfVectorizer(min_df=1)
tfidf = vect.fit_transform(corpus)
pairwise_similarity = tfidf * tfidf.T 

In [12]:
pd.DataFrame(pairwise_similarity.toarray())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.000000,0.344765,0.459007,0.404168,0.051872,0.0,0.412011,0.399542,0.091936,0.000000,0.069957,0.067480,0.072130,0.000000,0.000000
1,0.344765,1.000000,0.606862,0.429986,0.032721,0.0,0.869887,0.528241,0.057994,0.000000,0.044129,0.042566,0.045499,0.000000,0.000000
2,0.459007,0.606862,1.000000,0.572468,0.043563,0.0,0.897613,0.870448,0.077210,0.000000,0.058752,0.056671,0.060576,0.000000,0.000000
3,0.404168,0.429986,0.572468,1.000000,0.038358,0.0,0.513854,0.498303,0.067986,0.000000,0.051732,0.049900,0.053339,0.000000,0.000000
4,0.051872,0.032721,0.043563,0.038358,1.000000,0.0,0.039103,0.037919,0.000000,0.000000,0.139590,0.038735,0.041405,0.100683,0.089039
5,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.412011,0.869887,0.897613,0.513854,0.039103,0.0,1.000000,0.781325,0.069305,0.000000,0.052736,0.050869,0.054374,0.000000,0.000000
7,0.399542,0.528241,0.870448,0.498303,0.037919,0.0,0.781325,1.000000,0.067208,0.000000,0.051140,0.049329,0.052728,0.000000,0.000000
8,0.091936,0.057994,0.077210,0.067986,0.000000,0.0,0.069305,0.067208,1.000000,0.000000,0.000000,0.000000,0.000000,0.143156,0.126600
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.121350,0.000000,0.000000,0.122876,0.108665
